# **1.IMPORTING LIBRARIES**

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import os 
import pandas as pd 

import plotly.express as px
import datetime
import seaborn as sns
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

#**2.READING DATASET**

In [ ]:
df = pd.read_excel('~/downloads/Data.xlsx')
df.head(2)



In [ ]:
df.tail(2)


In [ ]:
df.shape

# **3.DATA CLEANING**

**Data Types**

In [ ]:
df.info()

**Checking Null Values**

In [ ]:
print(df.isnull().sum())


**Drop nan**

As they are very less so we drop the nan records from dataset.

In [ ]:
df1 = df.dropna(how='any',axis=0)
df1.reset_index(inplace=True) 

print(df1.shape)
print(df1.isnull().sum())



**Drop** **Duplicates**

In [ ]:
df2=df1.drop_duplicates(keep=False)
#df2.reset_index(inplace=True)
print(df2.shape)


**Categorical Features Summarized**

1.Top count features

2.Frequency

3.Number of unique labels

In [ ]:
def categorical_summarized(dataframe, x=None, y=None, hue=None, palette='Set1', verbose=True):
   
    if x == None:
        column_interested = y
    else:
        column_interested = x
    series = df2[column_interested]
    print(series.describe())
    print('mode: ', series.mode())
    if verbose:
        print('-'*40)
        print(series.value_counts())

        print('*'*80)

In [ ]:
categorical_summarized(df2, x='Vendor_Name', y=None, hue=None, palette='Set1', verbose=True)
categorical_summarized(df2, x='IP_Address', y=None, hue=None, palette='Set1', verbose=True)
categorical_summarized(df2, x='Date_Time', y=None, hue=None, palette='Set1', verbose=True)
categorical_summarized(df2, x='Type', y=None, hue=None, palette='Set1', verbose=True)

.**Convert DATE_TIME (D.TYPE=OBJECT) TO (D.TYPE=DATE**)

In [ ]:
df3=df2.copy() # BACK UP OF DF2
from dateutil.parser import parse
lst=[]
for i in df3.Date_Time:
  lst.append(pd.to_datetime(parse(i)))


df3['date']=pd.DataFrame(lst)  

In [ ]:
df3.info()

In [ ]:
df3.isnull().sum()

In [ ]:
df4=df3.copy()
df4["IP_Address"] = df4["IP_Address"].astype('category')
df4["IP_Address_En"] = df4["IP_Address"].cat.codes
df4["Vendor_Name"] = df4['Vendor_Name'].astype('category')
df4["Vendor_Name_En"] = df4["Vendor_Name"].cat.codes
df4["Type"] = df4['Type'].astype('category')
df4["Type_En"] = df4["Type"].cat.codes
df4.head(30)


In [ ]:
df4.tail(3)

In [ ]:
df5=df4[['date','IP_Address_En','Vendor_Name_En','Type_En']]
df5.head()

In [ ]:
df6=df5.copy()# back up of df5

Distribution of data

In [ ]:
for i in df6.columns:

  plt.hist(df6[i], bins=100)
  print(i)
  plt.show()

**Bivariate Analysis**

In [ ]:
grouping = df6.groupby('IP_Address_En')['Vendor_Name_En'].count().reset_index()

In [ ]:
fig = px.line(grouping, x='IP_Address_En', y="Vendor_Name_En", 
              title="Ip_Address over vender count")
fig.show()

In [ ]:
grouping = df6.groupby('IP_Address_En')['Type_En'].count().reset_index()

In [ ]:
fig = px.line(grouping, x='IP_Address_En', y="Type_En", 
              title="Ip_Address over Type count")
fig.show()

In [ ]:
grouping = df6.groupby('date')['IP_Address_En'].count().reset_index()
fig = px.line(grouping, x="date", y="IP_Address_En", 
              title="date over ip adress_count")
fig.show()

In [ ]:
grouping = df6.groupby('Vendor_Name_En')['IP_Address_En'].count().reset_index()


In [ ]:
fig = px.line(grouping, x="Vendor_Name_En", y="IP_Address_En", 
              title="Vendor_Name over ip adress_count")
fig.show()

In [ ]:
sns.countplot(x='Type', data=df2)
plt.title('TYpe count')
plt.show()

**Outliers Detection**

**ONE HOT ENCODING**

In [ ]:
df7=df4[['Type','date','IP_Address','Vendor_Name']]
df7.head(1)

In [ ]:

ip_address=pd.get_dummies(df7['IP_Address'])
len(ip_address.columns)

In [ ]:
from sklearn import preprocessing
scaled_data = preprocessing.scale(ip_address.T)
scaled_data.shape

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(scaled_data)

In [ ]:
pca_data = pca.transform(scaled_data)
pca_data.shape

In [ ]:
per_var=np.round(pca.explained_variance_ratio_* 100, decimals=1)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]
len(labels)

In [ ]:
pca_df = pd.DataFrame(pca_data, index=[ip_address.columns], columns=labels)
pca_df.head(2)

In [ ]:
plt.scatter(pca_df.PC1, pca_df.PC2)
plt.title('My PCA Graph')
plt.xlabel('PC1 - {0}%'.format(per_var[0]))
plt.ylabel('PC2 - {0}%'.format(per_var[1]))
 
for sample in pca_df.index:
    plt.annotate(sample, (pca_df.PC1.loc[sample], pca_df.PC2.loc[sample]))
 
plt.show()

In [ ]:
df8=pd.concat([df7.drop(['IP_Address'],axis='columns'),ip_address],axis='columns')
df8.head(2)

In [ ]:
to_model_columns=df8.columns[3:390]
from sklearn.ensemble import IsolationForest
clf=IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.12), \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)
clf.fit(df8[to_model_columns])
pred = clf.predict(df8[to_model_columns])
df8['anomaly']=pred
outliers=df8.loc[df8['anomaly']==-1]
outlier_index=list(outliers.index)
print(outlier_index)
#Find the number of anomalies and normal points here points classified -1 are anomalous
print(df8['anomaly'].value_counts())

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
pca = PCA(n_components=3)  # Reduce to k=3 dimensions
scaler = StandardScaler()
#normalize the metrics
X = scaler.fit_transform(df8[to_model_columns])
X_reduce = pca.fit_transform(X)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_zlabel("x_composite_3")
# Plot the compressed data points
ax.scatter(X_reduce[:, 0], X_reduce[:, 1], zs=X_reduce[:, 2], s=4, lw=1, label="inliers",c="green")
# Plot x's for the ground truth outliers
ax.scatter(X_reduce[outlier_index,0],X_reduce[outlier_index,1], X_reduce[outlier_index,2],
           lw=2, s=60, marker="x", c="red", label="outliers")
ax.legend()
plt.show()

In [ ]:
to_model_columns=df8.columns[3:390]
from sklearn.decomposition import PCA
pca = PCA(2)
pca.fit(df8[to_model_columns])
res=pd.DataFrame(pca.transform(df8[to_model_columns]))
Z = np.array(res)
plt.title("IsolationForest")
plt.contourf( Z, cmap=plt.cm.Blues_r)
b1 = plt.scatter(res[0], res[1], c='green',
                 s=20,label="normal points")
b1 =plt.scatter(res.iloc[outlier_index,0],res.iloc[outlier_index,1], c='green',s=20,  edgecolor="red",label="predicted outliers")
plt.legend(loc="upper right")
plt.show()

**Top 20 Best Vendors**

In [ ]:
df5=df4
temp_df = pd.DataFrame(df5['Vendor_Name'].value_counts()).head(20)
print(temp_df.head(2))
import os

x = list(temp_df.index)[::-1]
y = list(temp_df['Vendor_Name'])[::-1]

fig, ax = plt.subplots()    
width = 0.75 # the width of the bars 
ind = np.arange(len(y))  # the x locations for the groups
ax.barh(ind, y, width, color="c")
ax.set_yticks(ind+width/30)
ax.set_yticklabels(x, minor=False, fontsize=15)
plt.xticks([])

plt.rcParams['figure.figsize'] = (30,10)
plt.rcParams.update({'font.size': 15})
plt.title('BEST VENDOR', fontsize=35)
plt.xlabel('Total count per vendor', fontsize=20)
plt.ylabel('Vendor Name', fontsize=20)      
#plt.show()
plt.savefig(os.path.join('test.png'), dpi=300, format='png', bbox_inches='tight', color='c') 

for i, v in enumerate(y):
    ax.text(v + 1, i + -0.15, str(v), color='k', fontweight='bold', fontsize=20)

**Low Performer Vendors**

In [ ]:
temp_df = pd.DataFrame(df5['Vendor_Name'].value_counts()).tail(20)
print(temp_df.head(2))

x = list(temp_df.index)[::-1]
y = list(temp_df['Vendor_Name'])[::-1]

fig, ax = plt.subplots()    
width = 0.75 # the width of the bars 
ind = np.arange(len(y))  # the x locations for the groups
ax.barh(ind, y, width, color="c")
ax.set_yticks(ind+width/30)
ax.set_yticklabels(x, minor=False, fontsize=15)
plt.xticks([])

plt.rcParams['figure.figsize'] = (30,10)
plt.rcParams.update({'font.size': 15})
plt.title('LOW PERFORMANCE VENDORS', fontsize=35)
plt.xlabel('Total Count per Vendor', fontsize=20)
plt.ylabel('Vendor Name', fontsize=20)      
#plt.show()
plt.savefig(os.path.join('test.png'), dpi=300, format='png', bbox_inches='tight', color='c') 

for i, v in enumerate(y):
    ax.text(v + 1, i + -0.15, str(v), color='k', fontweight='bold', fontsize=20)

**Best DateWise**

In [ ]:
from tqdm import tqdm
from dateutil.parser import parse
lst=[]
for i in tqdm(df.Date_Time):
  lst.append(parse(i).strftime('%m/%d/%Y'))

In [ ]:
temp_df = df.copy()
temp_df['Date_Time'] = lst
temp_df['temp_num'] = [1 for i in range(temp_df.shape[0])]
temp_df.drop(['IP_Address', 'Vendor_Name', 'Type'], axis=1, inplace=True)
temp_df = pd.DataFrame(temp_df['temp_num'] ,index=temp_df['Date_Time'])
temp_df.fillna(1, inplace=True)
temp_df = temp_df.groupby(axis=0, level=0).sum()
temp_df

In [ ]:
import operator

temp_df = pd.DataFrame(sorted(dict(zip(list(temp_df.index), list(temp_df['temp_num']))).items(), reverse=True ,key=operator.itemgetter(1)), columns=['Date_Time', 'temp_num'])

x = list(temp_df['Date_Time'])[::-1]
y = list(temp_df['temp_num'])[::-1]
y = list(int(j) for j in y)

fig, ax = plt.subplots()    
width = 0.75 # the width of the bars 
ind = np.arange(len(y))  # the x locations for the groups
ax.barh(ind, y, width, color="c")
ax.set_yticks(ind+width/30)
ax.set_yticklabels(x, minor=False, fontsize=15)
plt.xticks([])

plt.rcParams['figure.figsize'] = (30,10)
plt.rcParams.update({'font.size': 15})
plt.title('Best Date Wise', fontsize=35)
plt.xlabel('Numbers', fontsize=20)
plt.ylabel('Date', fontsize=20)      
#plt.show()
plt.savefig(os.path.join('test.png'), dpi=300, format='png', bbox_inches='tight', color='c') 

for i, v in enumerate(y):
    ax.text(v + -1800, i + -0.15, str(v), color='w', fontweight='bold', fontsize=20)

   